In [1]:
import pandas as pd
import numpy as np
import os
import json
import mysql.connector as sql
import geopandas as gpd
from fuzzywuzzy import fuzz, process
import requests, zipfile, io

In [2]:
Mydb = sql.connect(host = "localhost",
                   user = "root",
                   password = "Choudhary@24",
                   database = "phonepe")
mycursor = Mydb.cursor(buffered=True)
cursor = Mydb.cursor()

In [3]:
# Export JSON data for Phonepe

#url = "https://github.com/PhonePe/pulse/archive/refs/heads/master.zip"
#extract_path = "D:/DSprojects/Phonepe/pulse_data"

#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall(extract_path)

#print("Repo downloaded and extracted successfully.")

In [4]:
# Use raw string or forward slashes
Path_1 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\aggregated\transaction\country\india\state"
agg_trans_list = os.listdir(Path_1)

columns_1 = {
    'State': [], 'Year': [], 'Quarter': [],
    'Transaction_type': [], 'Transaction_count': [], 'Transaction_amount': []
}

for state in agg_trans_list:
    cur_state = os.path.join(Path_1, state)
    if not os.path.isdir(cur_state):
        continue

    agg_year_list = os.listdir(cur_state)

    for year in agg_year_list:
        cur_year = os.path.join(cur_state, year)
        if not os.path.isdir(cur_year):
            continue

        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = os.path.join(cur_year, file)
            try:
                with open(cur_file, 'r') as f:
                    A = json.load(f)

                if 'data' in A and A['data'] and 'transactionData' in A['data']:
                    for i in A['data']['transactionData']:
                        name = i['name']
                        count = i['paymentInstruments'][0]['count']
                        amount = i['paymentInstruments'][0]['amount']

                        columns_1['Transaction_type'].append(name)
                        columns_1['Transaction_count'].append(count)
                        columns_1['Transaction_amount'].append(amount)
                        columns_1['State'].append(state)
                        columns_1['Year'].append(year)
                        columns_1['Quarter'].append(int(file.strip('.json')))
            except Exception as e:
                print(f"Failed to process {cur_file}: {e}")
                continue

df_agg_trans = pd.DataFrame(columns_1)
df_agg_trans.head()


,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05


In [5]:
Path_2 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\aggregated\user\country\india\state"
agg_user_list = os.listdir(Path_2)

columns_2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [], 'Percentage': []}
for state in agg_user_list:
    cur_state = os.path.join(Path_2, state)
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = os.path.join(cur_state, year)
        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = os.path.join(cur_year, file)
            with open(cur_file, 'r') as data:
                B = json.load(data)
                try:
                    for i in B["data"]["usersByDevice"]:
                        brand_name = i["brand"]
                        counts = i["count"]
                        percents = i["percentage"]
                        columns_2["Brands"].append(brand_name)
                        columns_2["Count"].append(counts)
                        columns_2["Percentage"].append(percents)
                        columns_2["State"].append(state)
                        columns_2["Year"].append(year)
                        columns_2["Quarter"].append(int(file.strip('.json')))
                except:
                    pass

df_agg_user = pd.DataFrame(columns_2)
df_agg_user

,State,Year,Quarter,Brands,Count,Percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


In [6]:
path_3 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\map\transaction\hover\country\india\state"
map_Tran_List = os.listdir(path_3)

columns_3 = {'State':[], 'Year':[], 'Quarter':[], 'District':[], 'Count':[], 'Amount':[]}

for state in map_Tran_List:
    cur_state = os.path.join(path_3, state)
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = os.path.join(cur_state, year)
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = os.path.join(cur_year, file)
            with open(cur_file, 'r') as data:
                c = json.load(data)
                
                for i in c["data"]["hoverDataList"]:
                    district = i["name"]
                    count = i["metric"][0]["count"]
                    amount = i["metric"][0]["amount"]
                    columns_3["District"].append(district)
                    columns_3["Count"].append(count)
                    columns_3["Amount"].append(amount)
                    columns_3['State'].append(state)
                    columns_3['Year'].append(year)
                    columns_3['Quarter'].append(int(file.strip('.json')))

Map_Trans = pd.DataFrame(columns_3)
Map_Trans

,State,Year,Quarter,District,Count,Amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
20599,west-bengal,2024,4,alipurduar district,15875637,2.099251e+10
20600,west-bengal,2024,4,paschim bardhaman district,56616799,6.968735e+10
20601,west-bengal,2024,4,nadia district,65274337,1.079320e+11
20602,west-bengal,2024,4,birbhum district,36905213,5.778701e+10


In [7]:
Path_4 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\map\user\hover\country\india\state"

map_user = os.listdir(Path_4)

Columns_4 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'RegisteredUser': [], 'AppOpens': []}

for state in map_user:
    cur_state = os.path.join(Path_4, state)
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = os.path.join(cur_state, year)
        map_list = os.listdir(cur_year)
        
        for file in map_list:
            cur_file = os.path.join(cur_year, file)
            with open(cur_file, 'r') as data:
                d = json.load(data)
                # Loop through the hoverData dictionary items
                for district, metrics in d["data"]["hoverData"].items():
                    registered_user = metrics["registeredUsers"]
                    app_opens = metrics['appOpens']
                    Columns_4["District"].append(district)
                    Columns_4["RegisteredUser"].append(registered_user)
                    Columns_4["AppOpens"].append(app_opens)
                    Columns_4['State'].append(state)
                    Columns_4['Year'].append(year)
                    Columns_4['Quarter'].append(int(file.strip('.json')))

data_map_User = pd.DataFrame(Columns_4)
data_map_User

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
20603,west-bengal,2024,4,alipurduar district,475688,31842355
20604,west-bengal,2024,4,paschim bardhaman district,1468252,80543469
20605,west-bengal,2024,4,nadia district,1861738,98740305
20606,west-bengal,2024,4,birbhum district,1114220,73465525


In [8]:
Path_5 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\top\transaction\country\india\state/"

top_trans_list = os.listdir(Path_5)

Columns_5 = {
    'State': [],
    'Year': [],
    'Quarter': [],
    'Pincode': [],
    'Transaction_count': [],
    'Transaction_amount': []
}

for state in top_trans_list:
    cur_state = Path_5 + state + "/"
    top_year_list = os.listdir(cur_state)
    
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        
        for file in top_file_list:
            cur_file = cur_year + file
            try:
                with open(cur_file, 'r') as data:
                    A = json.load(data)
                    
                for i in A.get('data', {}).get('pincodes', []):
                    name = i['entityName']
                    count = i['metric']['count']
                    amount = i['metric']['amount']
                    
                    Columns_5['Pincode'].append(name)
                    Columns_5['Transaction_count'].append(count)
                    Columns_5['Transaction_amount'].append(amount)
                    Columns_5['State'].append(state)
                    Columns_5['Year'].append(int(year))
                    Columns_5['Quarter'].append(int(file.strip('.json')))
            except (KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
                continue

# Create DataFrame
df_top_trans = pd.DataFrame(Columns_5)

# Optional: Ensure correct data types
df_top_trans['Transaction_count'] = pd.to_numeric(df_top_trans['Transaction_count'], errors='coerce')
df_top_trans['Transaction_amount'] = pd.to_numeric(df_top_trans['Transaction_amount'], errors='coerce')
df_top_trans['Year'] = pd.to_numeric(df_top_trans['Year'], errors='coerce')
df_top_trans['Quarter'] = pd.to_numeric(df_top_trans['Quarter'], errors='coerce')

# Show DataFrame
df_top_trans

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.769298e+06
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.238042e+06
2,andaman-&-nicobar-islands,2018,1,744102,969,3.519060e+06
3,andaman-&-nicobar-islands,2018,1,744105,685,1.298561e+06
4,andaman-&-nicobar-islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
9994,west-bengal,2024,4,711101,6753348,1.049483e+10
9995,west-bengal,2024,4,700059,6662715,8.009476e+09
9996,west-bengal,2024,4,700039,6575693,9.018748e+09
9997,west-bengal,2024,4,734001,6432082,8.996529e+09


In [9]:
Path_6 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\top\user\country\india\state/"
user_data = os.listdir(Path_6)
Columns_6 = {'State':[], 'Year':[], 'Quarter':[], 'Pincode':[], 'RegisteredUsers':[]}
for state in user_data:
    cur_state=Path_6 + state + "/"
    Year_data = os.listdir(cur_state)
    for year in Year_data:
        cur_year =cur_state + year + "/"
        Top_file = os.listdir(cur_year)
        for file in Top_file:
            cur_file = cur_year+file
            data = open(cur_file,'r')
            Data = json.load(data)
            for i in Data['data']['pincodes']:
                name = i['name']
                registeredUsers = i['registeredUsers']
                Columns_6['Pincode'].append(name)
                Columns_6['RegisteredUsers'].append(registeredUsers)
                Columns_6['State'].append(state)
                Columns_6['Year'].append(year)
                Columns_6['Quarter'].append(int(file.strip('.json')))
Top_user_Data = pd.DataFrame(Columns_6)
Top_user_Data

,State,Year,Quarter,Pincode,RegisteredUsers
0,andaman-&-nicobar-islands,2018,1,744103,1608
1,andaman-&-nicobar-islands,2018,1,744101,1108
2,andaman-&-nicobar-islands,2018,1,744105,1075
3,andaman-&-nicobar-islands,2018,1,744102,1006
4,andaman-&-nicobar-islands,2018,1,744104,272
...,...,...,...,...,...
9995,west-bengal,2024,4,733134,169596
9996,west-bengal,2024,4,700059,169448
9997,west-bengal,2024,4,711302,146034
9998,west-bengal,2024,4,700150,142151


In [10]:
#Data Cleaning and Explorations 
df_agg_trans['Transaction_amount'] = (df_agg_trans['Transaction_amount'] / 1e6).round(2)
df_agg_trans['Year'] = df_agg_trans['Year'].astype(int)
# Print head
print(df_agg_trans.head())

df_agg_user['Percentage'] = (df_agg_user['Percentage'] * 100).round(2)
df_agg_user['Year'] = df_agg_user['Year'].astype(int)
# Print head to confirm
print(df_agg_user.head())

Map_Trans['Amount'] = (Map_Trans['Amount'] / 1e6).round(2)
Map_Trans['Year'] = Map_Trans['Year'].astype(int)
# Print head
print(Map_Trans.head())

data_map_User['Year'] = data_map_User['Year'].astype(int)

df_top_trans.isna().sum()
missing_rows = df_top_trans[df_top_trans['Pincode'].isna()]
print(missing_rows)
df_top_trans.at[4810, 'Pincode'] = '194101'
df_top_trans.at[4852, 'Pincode'] = '194101'
df_top_trans['Pincode'] = df_top_trans['Pincode'].astype(int)
df_top_trans['Transaction_amount'] = (df_top_trans['Transaction_amount'] / 1e6).round(2)
df_top_trans



                       State  Year  Quarter          Transaction_type  \
0  andaman-&-nicobar-islands  2018        1  Recharge & bill payments   
1  andaman-&-nicobar-islands  2018        1     Peer-to-peer payments   
2  andaman-&-nicobar-islands  2018        1         Merchant payments   
3  andaman-&-nicobar-islands  2018        1        Financial Services   
4  andaman-&-nicobar-islands  2018        1                    Others   

   Transaction_count  Transaction_amount  
0               4200                1.85  
1               1871               12.14  
2                298                0.45  
3                 33                0.01  
4                256                0.18  
                       State  Year  Quarter   Brands  Count  Percentage
0  andaman-&-nicobar-islands  2018        1   Xiaomi   1665       24.70
1  andaman-&-nicobar-islands  2018        1  Samsung   1445       21.44
2  andaman-&-nicobar-islands  2018        1     Vivo    982       14.57
3  andaman-&-ni

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.77
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.24
2,andaman-&-nicobar-islands,2018,1,744102,969,3.52
3,andaman-&-nicobar-islands,2018,1,744105,685,1.30
4,andaman-&-nicobar-islands,2018,1,744104,340,1.04
...,...,...,...,...,...,...
9994,west-bengal,2024,4,711101,6753348,10494.83
9995,west-bengal,2024,4,700059,6662715,8009.48
9996,west-bengal,2024,4,700039,6575693,9018.75
9997,west-bengal,2024,4,734001,6432082,8996.53


In [11]:
#check Data Types
Top_user_Data.dtypes

State              object
Year               object
Quarter             int64
Pincode            object
RegisteredUsers     int64
dtype: object

In [12]:
df_top_trans.dtypes

State                  object
Year                    int64
Quarter                 int64
Pincode                 int64
Transaction_count       int64
Transaction_amount    float64
dtype: object

In [13]:
data_map_User.dtypes

State             object
Year               int64
Quarter            int64
District          object
RegisteredUser     int64
AppOpens           int64
dtype: object

In [14]:
Map_Trans.dtypes

State        object
Year          int64
Quarter       int64
District     object
Count         int64
Amount      float64
dtype: object

In [15]:
df_agg_user.dtypes

State          object
Year            int64
Quarter         int64
Brands         object
Count           int64
Percentage    float64
dtype: object

In [16]:
df_agg_trans.dtypes

State                  object
Year                    int64
Quarter                 int64
Transaction_type       object
Transaction_count       int64
Transaction_amount    float64
dtype: object

In [17]:
# Load GeoJSON files
state_gdf = gpd.read_file(r"D:/DSprojects/Phonepe/Geo_Json/india_state.geojson")
district_gdf = gpd.read_file(r"D:/DSprojects/Phonepe/Geo_Json/india_district.geojson")

# Extract unique state and district names using correct columns
geojson_states = state_gdf['NAME_1'].unique()
geojson_districts = district_gdf['NAME_2'].unique()

In [18]:
#Sync Mapping of State Names
# --- State normalization dictionary ---
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunachal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'delhi': 'Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}

# --- State cleaning function ---
def normalize_state(df):
    df['State'] = df['State'].str.lower().str.strip().replace(state_name_map)
    df['State'] = df['State'].str.title()
    return df

# --- Apply to all tables ---
df_agg_user = normalize_state(df_agg_user)
df_agg_trans = normalize_state(df_agg_trans)
Map_Trans = normalize_state(Map_Trans)
Map_User = normalize_state(data_map_User)
df_top_trans = normalize_state(df_top_trans)
Top_user_Data = normalize_state(Top_user_Data)

In [19]:
from fuzzywuzzy import fuzz

# Define fuzzy matching function
def match_district_by_words(district_name, geojson_districts, threshold=85):
    if pd.isna(district_name) or not isinstance(district_name, str):
        return district_name
    words = district_name.split()
    for geo_dist in geojson_districts:
        geo_words = geo_dist.split()
        for word in words:
            for geo_word in geo_words:
                if fuzz.ratio(word.lower(), geo_word.lower()) >= threshold:
                    return geo_dist  # return the matched geojson district name
    return district_name  # if no match, keep original

# Apply to both DataFrames
Map_Trans['District'] = Map_Trans['District'].apply(lambda x: match_district_by_words(x, geojson_districts))
Map_User['District'] = Map_User['District'].apply(lambda x: match_district_by_words(x, geojson_districts))


In [20]:
def count_unmatched(df, column_name, geojson_districts, threshold=85):
    unmatched_count = 0
    for dist in df[column_name].dropna().unique():
        if not isinstance(dist, str):
            continue
        matched = False
        words = dist.split()
        for geo_dist in geojson_districts:
            geo_words = geo_dist.split()
            for word in words:
                for geo_word in geo_words:
                    if fuzz.ratio(word.lower(), geo_word.lower()) >= threshold:
                        matched = True
                        break
                if matched:
                    break
            if matched:
                break
        if not matched:
            unmatched_count += 1
    return unmatched_count


In [21]:
print("Unmatched District Count (Map_Trans):", count_unmatched(Map_Trans, 'District', geojson_districts))
print("Unmatched District Count (Map_User):", count_unmatched(Map_User, 'District', geojson_districts))


Unmatched District Count (Map_Trans): 224
Unmatched District Count (Map_User): 224


In [22]:
#convert all dataframe to csv.
Map_Trans.to_csv("Map_Trans.csv",index=False)
Map_User.to_csv("Map_User.csv",index=False)
Top_user_Data.to_csv("Top_User.csv",index=False)
df_top_trans.to_csv("Top_Trans.csv",index=False)
df_agg_trans.to_csv("Agg_Trans.csv",index=False)
df_agg_user.to_csv("Agg_User.csv",index=False)

In [23]:
# Run each create table Statement
cursor.execute("DROP TABLE IF EXISTS map_trans")
cursor.execute("""CREATE TABLE map_trans (
    State VARCHAR(50), Year INT, Quarter INT, District VARCHAR(50),
    Count INT, Amount FLOAT)""")

cursor.execute("DROP TABLE IF EXISTS map_user")
cursor.execute("""CREATE TABLE map_user (
    State VARCHAR(50), Year INT, Quarter INT, District VARCHAR(50),
    RegisteredUser INT, AppOpens INT)""")
	
cursor.execute("DROP TABLE IF EXISTS top_user")
cursor.execute("""CREATE TABLE top_user (
    State VARCHAR(50), Year INT, Quarter INT, Pincode INT,
    RegisteredUsers INT)""")
	
cursor.execute("DROP TABLE IF EXISTS top_trans")
cursor.execute("""CREATE TABLE top_trans (
    State VARCHAR(50), Year INT, Quarter INT, Pincode INT,
    Transaction_count INT, Transaction_amount FLOAT)""")

cursor.execute("DROP TABLE IF EXISTS aggregated_trans")
cursor.execute("""CREATE TABLE aggregated_trans (
    State VARCHAR(50), Year INT, Quarter INT, Transaction_type VARCHAR(50),
    Transaction_count BIGINT, Transaction_amount FLOAT)""")

cursor.execute("DROP TABLE IF EXISTS aggregate_user")
cursor.execute("""CREATE TABLE aggregate_user (
    State VARCHAR(50), Year INT, Quarter INT, Brands VARCHAR(50),
    Count BIGINT, Percentage FLOAT)""")

Mydb.commit()


In [24]:
# Load all CSVs
map_trans = pd.read_csv("Map_Trans.csv")
map_user = pd.read_csv("Map_User.csv")
top_user = pd.read_csv("Top_User.csv")
top_trans = pd.read_csv("Top_Trans.csv")
agg_trans = pd.read_csv("Agg_Trans.csv")
agg_user = pd.read_csv("Agg_User.csv")

# Helper function to insert data
def insert_dataframe(df, table_name):
    for _, row in df.iterrows():
        columns = ', '.join(row.index)
        placeholders = ', '.join(['%s'] * len(row))
        sql_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cursor.execute(sql_query, tuple(row))
    Mydb.commit()
    print(f"✅ Inserted into `{table_name}`")

# Insert data into tables
insert_dataframe(map_trans, "map_trans")
insert_dataframe(map_user, "map_user")
insert_dataframe(top_user, "top_user")
insert_dataframe(top_trans, "top_trans")
insert_dataframe(agg_trans, "aggregated_trans")
insert_dataframe(agg_user, "aggregate_user")


✅ Inserted into `map_trans`
✅ Inserted into `map_user`
✅ Inserted into `top_user`
✅ Inserted into `top_trans`
✅ Inserted into `aggregated_trans`
✅ Inserted into `aggregate_user`
